# Airlines Price Discrimination Project — Part 1: Data Preparation and Cleaning
 
This notebook serves as the foundational step in a comprehensive analysis of airline price discrimination within the U.S. market. The primary objectives are to load, clean, and integrate airline flight and airport datasets, ensuring that each record is enriched with accurate city and state information for both origin and destination airports.

 
Specifically, this notebook:

1. Loads and inspects the raw airline flight dataset along with airport-level metadata.
2. Integrates origin and destination information, enriching each route with city, state, and geographic identifiers.
3. Cleans and standardizes data by handling missing, duplicate, and inconsistent airport codes.
4. Validates and exports a complete, reliable dataset for use in subsequent exploratory and econometric analysis.
 
The focus of this stage is data integrity and reproducibility — ensuring that the dataset accurately represents U.S. domestic routes and is suitable for modeling market structure, pricing behavior, and competition in the next phase [02_EDA.ipynb](02_EDA.ipynb).

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

### I.Load Main Ailrline Tickets Data

I load the main flight dataset from a pickle file. This dataset contains:

* Flight information (origin, destination, prices, etc.)
* Passenger details
* Route information
* Other flight characteristics needed for price discrimination analysis

In [2]:
data = pd.read_pickle("../data/Airline_tickets_data.pkl")
data.head()

,origin,finaldest,return,year,quarter,airports,return_sym,stops,avgprice,passengers,...,shareUS,shareVX,shareWN,shareXE,shareXJ,shareYV,shareYX,shareZW,treated,smallmkt
0,ABE,ABI,1,2010,2,7,1.0,2,530.0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,False,True
1,ABE,ABI,1,2010,3,7,0.0,2,822.5,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
2,ABE,ABI,1,2010,4,7,1.0,2,389.5,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
3,ABE,ABI,1,2011,1,7,1.0,2,381.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
4,ABE,ABI,1,2011,3,7,1.0,2,1096.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True


### II.Load Airports Data

Here I load the airport reference dataset which contains:

- Airport codes (IATA codes like 'LAX', 'JFK')
- Airport names
- City and state information
- Geographic coordinates

This data will be used to enrich the flight data with city and state information for both origin and destination airports.

In [3]:
airports = pd.read_csv("../data/Airports_data_raw.csv")
airports

,code,icao,name,latitude,longitude,elevation,url,time_zone,city_code,country,city,state,county,type
0,AAA,NTGA,Anaa,-17.350665,-145.511120,36,NaN,Pacific/Tahiti,AAA,PF,NaN,NaN,NaN,AP
1,AAB,YARY,Arrabury Airport,-26.696783,141.049092,328,NaN,Australia/Brisbane,AAB,AU,Tanbar,Queensland,Barcoo Shire,AP
2,AAC,HEAR,El Arish International Airport,31.074284,33.829172,85,NaN,Africa/Cairo,AAC,EG,Arish,Muhafazat Shamal Sina',NaN,AP
3,AAD,HCAD,Adado Airport,6.096286,46.637708,980,NaN,Africa/Khartoum,AAD,SO,Adado,NaN,NaN,AP
4,AAE,DABB,Les Salines Airport,36.821392,7.811857,36,NaN,Africa/Algiers,AAE,DZ,El Hadjar,Annaba,NaN,AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9798,ZZE,UBBZ,Zangilan International Airport,39.095122,46.733618,1604,NaN,Asia/Baku,ZZE,AZ,Karabakh,NaN,NaN,AP
9799,ZZN,NaN,Nationaltheatret RailS,59.913869,10.752245,0,NaN,Europe/Oslo,OSL,NO,Oslo,Oslo,Oslo,AP
9800,ZZO,UHSO,Zonalnoye,50.660627,142.772300,479,NaN,Asia/Sakhalin,ZZO,RU,Tymovskoye,Sakhalin,NaN,AP
9801,ZZU,FWUU,Mzuzu,-11.443044,34.011933,4115,NaN,Africa/Blantyre,ZZU,MW,Mzuzu,Northern Region,Mzimba District,AP


Separate origin and destination airport datasets to enable distinct merging of city and state info for each:

In [4]:
origin_airports = airports.copy()
origin_airports = origin_airports.rename(columns=lambda x: f"{x}_O") # O for origin

In [5]:
dest_airports = airports.copy()
dest_airports = dest_airports.rename(columns=lambda x: f"{x}_D") # D for destination

In [6]:
# merging airports data with data

data = data.merge(
    origin_airports,
    how='left',
    left_on='origin',
    right_on='code_O'
)

data = data.merge(
    dest_airports,
    how='left',
    left_on='finaldest',
    right_on='code_D'
)

data

,origin,finaldest,return,year,quarter,airports,return_sym,stops,avgprice,passengers,...,longitude_D,elevation_D,url_D,time_zone_D,city_code_D,country_D,city_D,state_D,county_D,type_D
0,ABE,ABI,1,2010,2,7,1.0,2,530.0,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
1,ABE,ABI,1,2010,3,7,0.0,2,822.5,2,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
2,ABE,ABI,1,2010,4,7,1.0,2,389.5,2,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
3,ABE,ABI,1,2011,1,7,1.0,2,381.0,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
4,ABE,ABI,1,2011,3,7,1.0,2,1096.0,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345074,YUM,YKM,1,2011,3,7,1.0,2,559.0,1,...,-120.536798,1099,NaN,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP
2345075,YUM,YKM,1,2014,2,7,1.0,2,959.0,1,...,-120.536798,1099,NaN,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP
2345076,YUM,YKM,1,2014,3,7,1.0,2,601.0,1,...,-120.536798,1099,NaN,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP
2345077,YUM,YKM,1,2016,2,7,1.0,2,420.0,1,...,-120.536798,1099,NaN,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP


Identify flights missing origin or destination city info:

In [ ]:
# rows where origin_city or destination_city is NaN (or both)
data[data['city_O'].isna() | data['city_D'].isna()]

,origin,finaldest,return,year,quarter,airports,return_sym,stops,avgprice,passengers,...,longitude_D,elevation_D,url_D,time_zone_D,city_code_D,country_D,city_D,state_D,county_D,type_D
0,ABE,ABI,1,2010,2,7,1.0,2,530.000000,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
1,ABE,ABI,1,2010,3,7,0.0,2,822.500000,2,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
2,ABE,ABI,1,2010,4,7,1.0,2,389.500000,2,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
3,ABE,ABI,1,2011,1,7,1.0,2,381.000000,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
4,ABE,ABI,1,2011,3,7,1.0,2,1096.000000,1,...,-99.679370,1791,NaN,America/Chicago,ABI,US,NaN,NaN,NaN,AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2344094,YUM,ROC,1,2015,3,7,0.0,2,506.000000,2,...,-77.665221,508,NaN,America/New_York,ROC,US,NaN,NaN,NaN,AP
2344095,YUM,ROC,1,2016,1,7,0.0,2,595.500000,2,...,-77.665221,508,NaN,America/New_York,ROC,US,NaN,NaN,NaN,AP
2344096,YUM,ROC,1,2016,2,7,0.0,2,849.000000,1,...,-77.665221,508,NaN,America/New_York,ROC,US,NaN,NaN,NaN,AP
2344097,YUM,ROC,1,2016,3,7,0.0,2,605.666687,3,...,-77.665221,508,NaN,America/New_York,ROC,US,NaN,NaN,NaN,AP


List all unique origin and destination airport codes that don't have city info:

In [8]:
# unique finaldest codes where destination_city is missing
print((data[data['city_O'].isna()]['origin'].unique()))

['ABI' 'ABY' 'ADK' 'ADQ' 'AGS' 'AKN' 'AMA' 'ANC' 'ATL' 'BET' 'BQK' 'BRW'
 'CAE' 'CDB' 'CDV' 'CIU' 'CLT' 'CRP' 'CSG' 'CYS' 'DCA' 'DLG' 'DUT' 'FAI'
 'FLL' 'FNL' 'GST' 'HOM' 'HOU' 'IPL' 'ISN' 'KOA' 'KTN' 'LAS' 'LBB' 'MAF'
 'MGM' 'MVY' 'OKC' 'OME' 'OTZ' 'PSG' 'ROC' 'SCC' 'SIT' 'SNP' 'TEX' 'WRG'
 'YAK']


In [9]:
# unique finaldest codes where destination_city is missing
print((data[data['city_D'].isna()]['finaldest'].unique()))

['ABI' 'ABY' 'AGS' 'AMA' 'ANC' 'ATL' 'BQK' 'CAE' 'CIU' 'CLT' 'CRP' 'CSG'
 'DCA' 'FAI' 'FLL' 'HOU' 'KOA' 'LAS' 'LBB' 'MAF' 'MGM' 'OKC' 'ROC' 'KTN'
 'ADQ' 'BET' 'CYS' 'ISN' 'SIT' 'WRG' 'HOM' 'ADK' 'AKN' 'BRW' 'CDV' 'DLG'
 'DUT' 'GST' 'OME' 'PSG' 'SNP' 'OTZ' 'SCC' 'YAK' 'MVY' 'TEX' 'FNL' 'CDB']


### III. Load Additional Airport Data

Since I have missing airport information, I load a second airport dataset that may contain additional airport codes and city information not available in the first dataset.


In [10]:
airports_add = pd.read_pickle("../data/Airports_data_raw_add.pkl")
airports_add

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3606798,STL,TBN,"St. Louis, MO","Fort Leonard Wood, MO",281,969,51,119,2009-02-01,2828990,46457,38.748699,-90.370003,37.741600,-92.140701
3606799,STL,TBN,"St. Louis, MO","Fort Leonard Wood, MO",245,1026,54,119,2009-11-01,2828990,46457,38.748699,-90.370003,37.741600,-92.140701
3606800,STL,TBN,"St. Louis, MO","Fort Leonard Wood, MO",363,1273,67,119,2009-08-01,2828990,46457,38.748699,-90.370003,37.741600,-92.140701
3606801,CGI,TBN,"Cape Girardeau, MO","Fort Leonard Wood, MO",2,19,1,146,2009-08-01,93712,46457,37.225300,-89.570801,37.741600,-92.140701


I separate the additional airport data into origin and destination components, similar to what I did with the first dataset. This allows me to merge this data with main dataset to fill in missing city info.


In [11]:
origin_add=airports_add[['Origin_airport', 'Origin_city']].drop_duplicates()
destin_add=airports_add[['Destination_airport', 'Destination_city']].drop_duplicates()

In [12]:
# combine the origin and destination data from the additional dataset into a single comprehensive airport reference table
airports_combined = pd.concat([
    origin_add.rename(columns={'Origin_airport': 'airport', 'Origin_city': 'city'}),
    destin_add.rename(columns={'Destination_airport': 'airport', 'Destination_city': 'city'})
], ignore_index=True).drop_duplicates(subset=['airport', 'city'])[['airport', 'city']]

Check that combined airport dataset doesn't contain duplicate rows:

In [13]:
print("Number of duplicate rows:", airports_combined.duplicated().sum())

Number of duplicate rows: 0


In [14]:
airports_combined

,airport,city
0,MHK,"Manhattan, KS"
1,EUG,"Eugene, OR"
2,MFR,"Medford, OR"
3,SEA,"Seattle, WA"
4,PDX,"Portland, OR"
...,...,...
1338,MIW,"Marshalltown, IA"
1364,JCC,"San Francisco, CA"
1371,NGP,"Corpus Christi, TX"
1376,MPS,"Mount Pleasant, TX"


In [15]:
# create columns 'city' and 'state'
airports_combined['state'] = airports_combined['city'].str.split(',').str[-1].str.strip()
airports_combined['city'] = airports_combined['city'].str.split(',').str[0].str.strip()
airports_combined

,airport,city,state
0,MHK,Manhattan,KS
1,EUG,Eugene,OR
2,MFR,Medford,OR
3,SEA,Seattle,WA
4,PDX,Portland,OR
...,...,...,...
1338,MIW,Marshalltown,IA
1364,JCC,San Francisco,CA
1371,NGP,Corpus Christi,TX
1376,MPS,Mount Pleasant,TX


Prepare Additional origin and destination airport data

In [16]:
airports_or = airports_combined.copy()
airports_or= airports_or.rename(columns=lambda x: f"{x}_O1") # O for origin
airports_or

,airport_O1,city_O1,state_O1
0,MHK,Manhattan,KS
1,EUG,Eugene,OR
2,MFR,Medford,OR
3,SEA,Seattle,WA
4,PDX,Portland,OR
...,...,...,...
1338,MIW,Marshalltown,IA
1364,JCC,San Francisco,CA
1371,NGP,Corpus Christi,TX
1376,MPS,Mount Pleasant,TX


In [17]:
airports_de = airports_combined.copy()
airports_de= airports_de.rename(columns=lambda x: f"{x}_D1") # D for destination
airports_de

,airport_D1,city_D1,state_D1
0,MHK,Manhattan,KS
1,EUG,Eugene,OR
2,MFR,Medford,OR
3,SEA,Seattle,WA
4,PDX,Portland,OR
...,...,...,...
1338,MIW,Marshalltown,IA
1364,JCC,San Francisco,CA
1371,NGP,Corpus Christi,TX
1376,MPS,Mount Pleasant,TX


### VI. Merge Additional Origin Airport Data

Here I merge the additional origin airport data with main dataset to fill in missing city information for origin airports


In [18]:
data = data.merge(airports_or, left_on='origin', right_on='airport_O1', how='left')
data

,origin,finaldest,return,year,quarter,airports,return_sym,stops,avgprice,passengers,...,time_zone_D,city_code_D,country_D,city_D,state_D,county_D,type_D,airport_O1,city_O1,state_O1
0,ABE,ABI,1,2010,2,7,1.0,2,530.0,1,...,America/Chicago,ABI,US,NaN,NaN,NaN,AP,ABE,Allentown,PA
1,ABE,ABI,1,2010,3,7,0.0,2,822.5,2,...,America/Chicago,ABI,US,NaN,NaN,NaN,AP,ABE,Allentown,PA
2,ABE,ABI,1,2010,4,7,1.0,2,389.5,2,...,America/Chicago,ABI,US,NaN,NaN,NaN,AP,ABE,Allentown,PA
3,ABE,ABI,1,2011,1,7,1.0,2,381.0,1,...,America/Chicago,ABI,US,NaN,NaN,NaN,AP,ABE,Allentown,PA
4,ABE,ABI,1,2011,3,7,1.0,2,1096.0,1,...,America/Chicago,ABI,US,NaN,NaN,NaN,AP,ABE,Allentown,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345074,YUM,YKM,1,2011,3,7,1.0,2,559.0,1,...,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ
2345075,YUM,YKM,1,2014,2,7,1.0,2,959.0,1,...,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ
2345076,YUM,YKM,1,2014,3,7,1.0,2,601.0,1,...,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ
2345077,YUM,YKM,1,2016,2,7,1.0,2,420.0,1,...,America/Los_Angeles,YKM,US,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ


Rows where 'city_O' is missing but 'city_O1' can fill it:

In [19]:
data.loc[data['city_O'].isna() & data['city_O1'].notna(), ['origin', 'city_O', 'city_O1']].drop_duplicates(subset=['origin'])

,origin,city_O,city_O1
8213,ABI,NaN,Abilene
29418,ABY,NaN,Albany
44247,ADQ,NaN,Kodiak
53951,AGS,NaN,Augusta
78015,AMA,NaN,Amarillo
86041,ANC,NaN,Anchorage
105726,ATL,NaN,Atlanta
287694,BQK,NaN,Brunswick
365030,CAE,NaN,Columbia
445161,CLT,NaN,Charlotte


These are rows that are still missing information about the origin city; they may need to be populated manually if necessary:

In [20]:
data.loc[data['city_O'].isna() & data['city_O1'].isna(), ['origin', 'city_O', 'city_O1']].drop_duplicates(subset=['origin'])

,origin,city_O,city_O1
44140,ADK,NaN,NaN
62308,AKN,NaN,NaN
181063,BET,NaN,NaN
299628,BRW,NaN,NaN
384216,CDB,NaN,NaN
385267,CDV,NaN,NaN
424089,CIU,NaN,NaN
625066,DLG,NaN,NaN
664918,DUT,NaN,NaN
819506,FNL,NaN,NaN


The same steps for destination airports:

In [21]:
data = data.merge(airports_de, left_on='finaldest', right_on='airport_D1', how='left')
data

,origin,finaldest,return,year,quarter,airports,return_sym,stops,avgprice,passengers,...,city_D,state_D,county_D,type_D,airport_O1,city_O1,state_O1,airport_D1,city_D1,state_D1
0,ABE,ABI,1,2010,2,7,1.0,2,530.0,1,...,NaN,NaN,NaN,AP,ABE,Allentown,PA,ABI,Abilene,TX
1,ABE,ABI,1,2010,3,7,0.0,2,822.5,2,...,NaN,NaN,NaN,AP,ABE,Allentown,PA,ABI,Abilene,TX
2,ABE,ABI,1,2010,4,7,1.0,2,389.5,2,...,NaN,NaN,NaN,AP,ABE,Allentown,PA,ABI,Abilene,TX
3,ABE,ABI,1,2011,1,7,1.0,2,381.0,1,...,NaN,NaN,NaN,AP,ABE,Allentown,PA,ABI,Abilene,TX
4,ABE,ABI,1,2011,3,7,1.0,2,1096.0,1,...,NaN,NaN,NaN,AP,ABE,Allentown,PA,ABI,Abilene,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345074,YUM,YKM,1,2011,3,7,1.0,2,559.0,1,...,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ,YKM,Yakima,WA
2345075,YUM,YKM,1,2014,2,7,1.0,2,959.0,1,...,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ,YKM,Yakima,WA
2345076,YUM,YKM,1,2014,3,7,1.0,2,601.0,1,...,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ,YKM,Yakima,WA
2345077,YUM,YKM,1,2016,2,7,1.0,2,420.0,1,...,Yakima,Washington,Yakima County,AP,YUM,Yuma,AZ,YKM,Yakima,WA


Rows where 'city_D' is missing but 'city_D1' can fill it:

In [22]:
data.loc[data['city_D'].isna() & data['city_D1'].notna(), ['finaldest', 'city_D', 'city_D1']].drop_duplicates(subset=['finaldest'])

,finaldest,city_D,city_D1
0,ABI,NaN,Abilene
64,ABY,NaN,Albany
122,AGS,NaN,Augusta
177,AMA,NaN,Amarillo
194,ANC,NaN,Anchorage
252,ATL,NaN,Atlanta
864,BQK,NaN,Brunswick
1122,CAE,NaN,Columbia
1475,CLT,NaN,Charlotte
1636,CRP,NaN,Corpus Christi


These are rows that are still missing information about the destination city; they may need to be populated manually if necessary:

In [23]:
data.loc[data['city_D'].isna() & data['city_D1'].isna(), ['finaldest', 'city_D', 'city_D1']].drop_duplicates(subset=['finaldest'])

,finaldest,city_D,city_D1
1408,CIU,NaN,NaN
4005,KOA,NaN,NaN
15179,BET,NaN,NaN
25643,SIT,NaN,NaN
27018,WRG,NaN,NaN
44198,HOM,NaN,NaN
44247,ADK,NaN,NaN
44254,AKN,NaN,NaN
44533,BRW,NaN,NaN
44599,CDV,NaN,NaN


In [24]:
orig_missing = data.loc[data['city_O'].isna() & data['city_O1'].isna(), ['origin', 'city_O', 'city_O1']].drop_duplicates(subset=['origin'])
dest_missing = data.loc[data['city_D'].isna() & data['city_D1'].isna(), ['finaldest', 'city_D', 'city_D1']].drop_duplicates(subset=['finaldest'])
merged_missing = orig_missing.merge(dest_missing, left_on='origin', right_on='finaldest', how='outer', indicator=True)
unique_missing = merged_missing.drop_duplicates()
airport_missing = unique_missing[['origin']].rename(columns={'origin': 'airport'})
#airport_missing.to_csv("data/airport_missing.csv", index=False)

I use the additional airport data to fill in missing city information for all airports (origin and destination):

In [25]:
data['city_O'] = data['city_O'].fillna(data['city_O1'])
data['city_D'] = data['city_D'].fillna(data['city_D1'])

### V. Check Final Dataset Structure

I examine the columns in my final dataset to understand what data we have available for analysis.
All data from city_O1 and city_D1 was successfully populated into city_O and city_D:


In [26]:
print("Missing origin airports:")
print(data.loc[data['city_O'].isna() & data['city_O1'].notna(), ['origin', 'city_O', 'city_O1']].drop_duplicates(subset=['origin']))
print()
print(len(data.loc[data['city_O'].isna() & data['city_O1'].notna(), ['origin', 'city_O', 'city_O1']].drop_duplicates(subset=['origin'])))

Missing origin airports:
Empty DataFrame
Columns: [origin, city_O, city_O1]
Index: []

0


In [27]:
print("Missing destination airports that can be populated:")
print(data.loc[data['city_D'].isna() & data['city_D1'].notna(), ['finaldest', 'city_D', 'city_D1']].drop_duplicates(subset=['finaldest']))
print()
print(len(data.loc[data['city_D'].isna() & data['city_D1'].notna(), ['finaldest', 'city_D', 'city_D1']].drop_duplicates(subset=['finaldest'])))

Missing destination airports that can be populated:
Empty DataFrame
Columns: [finaldest, city_D, city_D1]
Index: []

0


### VI. Analyze Data Completeness

I analyze how much of our data is still missing city information after all our data merging and filling attempts. This helps us understand the quality of final dataset and identify any remaining data issues.

In [28]:
# calculate and print the % of rows missing city_O, city_D, or both
total_rows = len(data)
missing_city_O = data['city_O'].isna()
missing_city_D = data['city_D'].isna()
missing_both = missing_city_O | missing_city_D

num_missing_city_O = missing_city_O.sum()
num_missing_city_D = missing_city_D.sum()
num_missing_both = missing_both.sum()

print(f"Rows missing Origin city {num_missing_city_O} ({num_missing_city_O / total_rows:.2%})")
print(f"Rows missing Destination City: {num_missing_city_D} ({num_missing_city_D / total_rows:.2%})")
print(f"Rows missing both Origin and Destination cities: {num_missing_both} ({num_missing_both / total_rows:.2%})")

Rows missing Origin city 21930 (0.94%)
Rows missing Destination City: 26968 (1.15%)
Rows missing both Origin and Destination cities: 47096 (2.01%)


In [29]:
# calculate and print the percentage of distinct airports missing city_O, city_D, or both

# for origin airports (city_O)
origins = data['origin'].unique()
missing_city_O_airports = data.loc[data['city_O'].isna(), 'origin'].unique()
num_missing_city_O_airports = len(missing_city_O_airports)
total_origin_airports = len(origins)
pct_missing_city_O_airports = num_missing_city_O_airports / total_origin_airports

# for destination airports (city_D)
finaldests = data['finaldest'].unique()
missing_city_D_airports = data.loc[data['city_D'].isna(), 'finaldest'].unique()
num_missing_city_D_airports = len(missing_city_D_airports)
total_finaldest_airports = len(finaldests)
pct_missing_city_D_airports = num_missing_city_D_airports / total_finaldest_airports

# for all unique airports (origin or destination)
all_airports = set(data['origin']).union(set(data['finaldest']))
missing_city_airports = set(missing_city_O_airports).union(set(missing_city_D_airports))
num_missing_city_airports = len(missing_city_airports)
total_unique_airports = len(all_airports)
pct_missing_city_airports = num_missing_city_airports / total_unique_airports

print(f"Distinct origin airports missing Origin City: {num_missing_city_O_airports} of {total_origin_airports} ({pct_missing_city_O_airports:.2%})")
print(f"Distinct destination airports missing Destination city: {num_missing_city_D_airports} of {total_finaldest_airports} ({pct_missing_city_D_airports:.2%})")
print(f"Distinct airports (Origin or Destination) missing cities info: {num_missing_city_airports} of {total_unique_airports} ({pct_missing_city_airports:.2%})")

Distinct origin airports missing Origin City: 23 of 401 (5.74%)
Distinct destination airports missing Destination city: 23 of 400 (5.75%)
Distinct airports (Origin or Destination) missing cities info: 23 of 402 (5.72%)


In [30]:
# Calculate and print the number and percentage of unique routes missing city_O, city_D, or both

# Define a route as a tuple of (origin, finaldest)
routes = data[['origin', 'finaldest']].drop_duplicates()
total_routes = len(routes)

# Find routes missing city_O
routes_missing_city_O = data.loc[data['city_O'].isna(), ['origin', 'finaldest']].drop_duplicates()
num_routes_missing_city_O = len(routes_missing_city_O)
pct_routes_missing_city_O = num_routes_missing_city_O / total_routes

# Find routes missing city_D
routes_missing_city_D = data.loc[data['city_D'].isna(), ['origin', 'finaldest']].drop_duplicates()
num_routes_missing_city_D = len(routes_missing_city_D)
pct_routes_missing_city_D = num_routes_missing_city_D / total_routes

# Find routes missing either city_O or city_D (union)
routes_missing_either = pd.concat([routes_missing_city_O, routes_missing_city_D]).drop_duplicates()
num_routes_missing_either = len(routes_missing_either)
pct_routes_missing_either = num_routes_missing_either / total_routes

print(f"Unique routes missing Origin City: {num_routes_missing_city_O} of {total_routes} ({pct_routes_missing_city_O:.2%})")
print(f"Unique routes missing Destination City: {num_routes_missing_city_D} of {total_routes} ({pct_routes_missing_city_D:.2%})")
print(f"Unique routes missing either Origin or Destination City: {num_routes_missing_either} of {total_routes} ({pct_routes_missing_either:.2%})")

Unique routes missing Origin City: 997 of 64960 (1.53%)
Unique routes missing Destination City: 1194 of 64960 (1.84%)
Unique routes missing either Origin or Destination City: 2096 of 64960 (3.23%)


#### Explanation of missing city information

In the previous cells, I calculated how many rows in the dataset are missing city information for either the Origin city (city_O), the Destination city (city_D), or both.
For example, 21,930 out of ~2.34 million rows are missing Origin city (≈0.94%), and 26,068 rows are missing Destination city (≈1.15%). In total, 47,096 rows (≈2.01%) are missing either or both city fields.

I also examined the percentage of unique airports (for both origins and destinations) that lack city data. Out of roughly 400 unique airports, 23 airports (≈5.7%) are missing city information — indicating that the issue is concentrated in a small subset of airports.

Finally, I checked unique routes (distinct origin–destination pairs) to see how many are missing city information.
Out of 64,960 unique routes:

* 997 routes (≈1.53%) are missing the Origin city
* 1,194 routes (≈1.84%) are missing the Destination city
* 2,096 routes (≈3.23%) are missing either the Origin or Destination city

This confirms that m_issing city data is relatively rare but slightly more noticeable when aggregated at the route level.

---

**Is this data quality acceptable?**

Overall, yes. The percentage of missing city data is small across both the observation and route levels, making the dataset reliable for most analyses. However:

* If city-level analysis is critical, these missing values could bias results for certain airports or routes, especially if the missing airports are not missing at random (e.g., if they are mostly small or remote airports).
* For most aggregate analyses, this level of missingness is generally acceptable, but it's important to document it and consider sensitivity checks or imputation if city information is important for your analysis.


The dataset maintains high completeness — over 97% of routes and 98% of records contain valid city information. This level of data quality is sufficient for aggregate, route-level, and econometric analyses, as long as the small fraction of missing city data is transparently reported.

In [31]:
# drop NaNs rows and save dataset
data = data.dropna(subset=['city_O', 'city_D'])

In [32]:
# data.to_pickle('data/Airlines_ticket_data_ready.pkl')

In [ ]:
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaFileUpload
# import os

# def upload_to_drive(filepath):
#     SCOPES = ['https://www.googleapis.com/auth/drive.file']
    
#     # Authenticate
#     flow = InstalledAppFlow.from_client_secrets_file(
#         'credentials.json', SCOPES)
#     creds = flow.run_local_server(port=0)
    
#     # Build the Drive API service
#     service = build('drive', 'v3', credentials=creds)

#     # File metadata
#     file_metadata = {
#         'name': os.path.basename(filepath)
#     }

#     media = MediaFileUpload(filepath, resumable=True)

#     uploaded_file = service.files().create(
#         body=file_metadata,
#         media_body=media,
#         fields='id, webViewLink'
#     ).execute()

#     print("Upload complete!")
#     print("File ID:", uploaded_file.get('id'))
#     print("View:", uploaded_file.get('webViewLink'))

# # Example: Run this with your actual file
# upload_to_drive("data/Airline_tickets_data_ready.pkl")

### Summary & Next Steps

The flight and airport data were merged, cleaned, and standardized to ensure accurate origin–destination details and resolve missing or inconsistent airport codes. Duplicates and irrelevant records were removed, resulting in a reliable, analysis-ready dataset with complete route-level, pricing, and competition information.

Next Steps:

* Conduct exploratory data analysis (EDA) to examine fare distributions, market concentration, and carrier competition patterns.
* Validate data patterns using visualizations and correlation analysis to uncover initial insights about pricing dynamics.
* Prepare for econometric modeling in the next phase by testing key hypotheses around competition intensity, carrier dominance, and market structure [02_EDA.ipynb](02_EDA.ipynb).